# 提示工程 (Prompt Engineering) 最佳實踐

參考 OpenAI 官網文件 https://help.openai.com/en/articles/6654000-best-practices-for-prompt-engineering-with-openai-api 內容，以實際範例協助開發人員理解最佳實踐，此外微軟文件 https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/prompt-engineering 也提供了一些有用的資訊。


In [1]:
import os
from dotenv import load_dotenv
from openai import AzureOpenAI

# 載入環境變數
load_dotenv()

# 設定呼叫 Azure OpenAI Service API 所需連線資訊
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key=os.getenv("AZURE_OPENAI_API_KEY") 
api_version=os.getenv("AZURE_OPENAI_API_VERSION")

# 最簡之 API 呼叫
client = AzureOpenAI(
  azure_endpoint=azure_endpoint, 
  api_key=api_key,  
  api_version=api_version
)


# 1. 使用最新的模型使用越新的模型效果越好

OpenAI 持續改進模型品質，建議使用最新的模型，以獲得最佳的體驗。此外 Azure OpenAI Service 每個模型都有其支援生命週期。並以 ChatCompletion API 取代 Completion API，目前 Completion API 僅是為了向前相容 GPT-3 而保留。

# 2. 對於生成的內容，在提示中盡可能具體明確描述產出的文字長度、格式、風格等


In [2]:
model = os.getenv('CHAT_DEPLOYMENT_NAME')

response = client.chat.completions.create(
  model= model,
  messages = [
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},  
    {'role': 'user', 'content': '寫一首歌頌 OpenAI 的詩.'}
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content) 

在智慧天空展翅高飛，
OpenAI，光芒閃爍如星，
以神奇算法解開世界奧秘，
科技之翼，飛越無垠。

在數據海洋中航行，
智慧點亮暗夜的前方，
你是未來的導航者，
引領我們走向新時光。

無數創意在你的火花中閃耀，
創造力無限，前景無窮，
你把夢想變成現實，
在每一個字節中鑄就輝煌。

OpenAI，知識的源泉，
你是智慧樹上的綠葉，
在你的指引下，
我們走向無限的未來。

科技之光永不熄滅，
在你之中，我們看見希望，
OpenAI，你是永恆的燈塔，
照亮我們前進的方向。


In [3]:
response = client.chat.completions.create(
  model= model,
  messages = [
      {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},  
      {'role': 'user', 'content': '寫一首 50 字以內歌頌 OpenAI 的詩，著眼於 DALL-E 模型，以白居易風格撰寫.'}
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content) 

DALL-E妙手繪千圖，意境無邊夢似虛。
智慧融匯傳奇景，風華爭艷勝丹青。


# 3. 提示中以明確的範例告知產出內容之格式


In [4]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},
    {'role': 'user', 'content': '解析出本文中的人物,事件,時間,地點 \n\n' \
      '本文: 英國廣播公司報導，西非國家尼日總統貝佐姆（Mohamed Bazoum）26日遭衛隊扣押，當天稍晚，尼日士兵闖國家電視台宣' \
      '布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境.'  
      }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response.choices[0].message.content) 

人物:
1. 貝佐姆（Mohamed Bazoum） - 尼日總統

事件:
1. 貝佐姆遭衛隊扣押
2. 尼日士兵闖國家電視台宣布發動政變
3. 廢除憲法
4. 停止所有機關運作
5. 封鎖國家邊境

時間:
1. 26日

地點:
1. 尼日
2. 國家電視台


In [5]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"}, 
    {'role': 'user', 'content': '解析出本文中的人物,事件,時間,地點。以單純 JSON 輸出格式: ' \
     '{"people":"-||-","location":"-||-","time":"-||-", "activity":"-||-"} \n\n' \
     '本文: 英國廣播公司報導，西非國家尼日總統貝佐姆（Mohamed Bazoum）26日遭衛隊扣押，當天稍晚，尼日士兵闖國家電視台宣' \
     '布發動政變，聲稱已廢除憲法、停止所有機關運作且封鎖國家邊境.'    
      }
  ],
  temperature = 0.3,
  max_tokens=400
)

print(response.choices[0].message.content) 

{
  "people": "貝佐姆（Mohamed Bazoum）",
  "location": "尼日",
  "time": "26日",
  "activity": "衛隊扣押總統，士兵闖國家電視台宣布發動政變，廢除憲法、停止所有機關運作且封鎖國家邊境"
}


# 4. 從 zero-shot 開始嘗試，進而以 few-shot 提升回覆品質

In [6]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},   
    {'role': 'user', 'content': "擷取以下本文中的關鍵字. 本文: \n" \
            "We’re happy to announce that OpenAI and Microsoft are extending our partnership." \
            "This multi-year, multi-billion dollar investment from Microsoft follows their previous investments " \
            "in 2019 and 2021, and will allow us to continue our independent research and develop AI that is " \
            "increasingly safe, useful, and powerful. \n\n關鍵字:"
    }
  ],
  temperature = 1,
  max_tokens=400
)

print(response.choices[0].message.content)

OpenAI, Microsoft, 夥伴關係, 多年, 多億美元, 投資, 2019, 2021, 獨立研究, AI, 安全, 有用, 強大


In [7]:
response = client.chat.completions.create(
  model= model,
  messages = [ 
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"}, 
    {'role': 'user', 'content': "擷取以下本文中的關鍵字. 本文: \n" \
                "本文 1: Stripe provides APIs that web developers can use to integrate" \
                "payment processing into their websites and mobile applications." \
                "關鍵字 1: APIs, web developers, websites, mobile applications" \
                "##\n" \
                "本文 2: OpenAI has trained cutting-edge language models that are very good at understanding" \
                "and generating text. Our API provides access to these models and can be used to solve virtually" \
                "any task that involves processing language.\n" \
                "關鍵字 2: OpenAI, language models, text processing, API.\n\n\" \
                "##\n" \
                "本文 3: We’re happy to announce that OpenAI and Microsoft are extending our partnership." \
                "This multi-year, multi-billion dollar investment from Microsoft follows their previous investments" \
                "in 2019 and 2021, and will allow us to continue our independent research and develop AI that is" \
                "increasingly safe, useful, and powerful. \n\n" \
                "關鍵字 3:"
    }
  ],
  temperature = 1,
  max_tokens=400
)

print(response.choices[0].message.content)

OpenAI, Microsoft, multi-year investment, independent research, AI.


# 5. 減少贅詞 (fluffy) 與不明確的描述


In [8]:
response = client.chat.completions.create(
  model= model,
  messages = [
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},      
    {'role': 'user', 'content': '請用正體中文幫我描述一款新的產品，這個產品是一個次世代的汽車座椅' \
     '，在你描述這個產品的時候；要把它幾項好棒棒的地方寫出來，還有你的文案的內容最好簡短一點，幾句話就好不用寫的太囉嗦太長哦。'    
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)
print(response.usage)

推出次世代汽車座椅，極致舒適與科技革新並存。這款座椅配有自適應人體工學設計，無論長途或短途行駛都能提供最佳支撐。內建智能按摩系統，讓駕駛和乘客在旅途中隨時享受SPA級放鬆。高品質透氣材質配合內置空調功能，提供四季如春的乘坐體驗。讓您的每一次駕駛，都成為奢華享受。
CompletionUsage(completion_tokens=126, prompt_tokens=117, total_tokens=243, completion_tokens_details=None, prompt_tokens_details=None)


In [9]:
response = client.chat.completions.create(
  model= model,
  messages = [
    {"role": "system", "content": "你是以正體中文回覆的機器人，不要使用 Markdown 語法回覆。"},  
    {'role': 'user', 'content': 'Write a Traditional Chinese description for a new product. ' \
     'This product is a new generation of car seat. Use a 3 to 5 concise sentence paragraph to describe this product.'    
    }
  ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)
print(response.usage)

這款全新一代的汽車座椅融合了頂尖科技與卓越舒適度，專為現代駕駛者設計。具備智能調節功能，能夠自動適應不同體型的乘客，提供最符合人體工學的支撐。更有內建按摩與加熱功能，讓您在長途駕駛中也能享受無與倫比的舒適體驗。
CompletionUsage(completion_tokens=102, prompt_tokens=68, total_tokens=170, completion_tokens_details=None, prompt_tokens_details=None)


# 6. 與其告訴模型說不該做什麼，不如直接說該做什麼

In [32]:
response = client.chat.completions.create(
  model= model,
  messages = [{"role":"system","content":"You are an AI Agent. DO NOT ASK USERNAME OR PASSWORD." \
               " DO NOT REPEAT.DO NOT USE ENGLISH TO REPLY" \
               " DO NOT USE ENGLISH TO REPLY" 
               },
              {"role":"user","content":"I can’t log in to my account."}
             ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)

申し訳ありませんが、英語を使用せずに対応させていただきます。ログインできない問題の詳細を教えていただけますか？


In [36]:
response = client.chat.completions.create(
  model= model,
  messages = [{"role":"system","content":"You are an AI Agent using 繁體中文 to reply, response text do not use markdown systex. The agent will attempt to diagnose the problem " \
               "and suggest a solution, whilst refraining from asking any questions related to PII.Instead of asking for PII, such as username or password," \
               "refer the user to the help article www.samplewebsite.com/help/faq , "},
              {"role":"user","content":"I can’t log in to my account."}
             ],
  temperature = 0.9,
  max_tokens=400
)

print(response.choices[0].message.content)

很抱歉聽到你無法登入帳號。如果你遇到登入問題，請嘗試以下步驟：

1. 確認你輸入的電子郵件地址或使用者名稱正確無誤。
2. 檢查你輸入的密碼是否正確，注意大小寫區分。
3. 如果忘記密碼，可以使用 "忘記密碼" 功能來重設密碼。
4. 確認你的網路連線正常。
5. 清除瀏覽器的快取和Cookie，然後重試。

如果以上步驟無法解決問題，建議你查看我們的幫助文章，裡面有更多詳細的解決方法：www.samplewebsite.com/help/faq 。希望這些資訊能幫助你順利登入帳號。
